<a href="https://colab.research.google.com/github/sarathchandrikak/Machine-Learning/blob/main/Copy_of_textrank.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Text Rank Algorithm

In [1]:
! pip install rouge-score
! pip install datasets

  Preparing metadata (setup.py) ... done
  Created wheel for rouge-score: filename=rouge_score-0.1.2-py3-none-any.whl size=24933 sha256=23f87614d70de9149d425e5b10d89b49cbe0dd643966710bc6e9ea843079fa32
  Stored in directory: /root/.cache/pip/wheels/5f/dd/89/461065a73be61a532ff8599a28e9beef17985c9e9c31e541b4
Successfully built rouge-score
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 521.2/521.2 kB 8.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 13.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 16.3 MB/s eta 0:00:00


In [2]:
! pip install rouge_score

In [3]:
import numpy as np
import pandas as pd
import nltk
nltk.download('punkt')
import re
from nltk.tokenize import sent_tokenize
from sklearn.metrics.pairwise import cosine_similarity
from rouge_score import rouge_scorer

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [4]:
from sklearn.metrics.pairwise import cosine_similarity
import networkx as nx

In [5]:
from datasets import load_dataset
dataset = load_dataset("multi_news")

Generating train split:   0%|          | 0/44972 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/5622 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/5622 [00:00<?, ? examples/s]

In [6]:
train_df = pd.DataFrame.from_dict(dataset['train'])
train_df.shape

(44972, 2)

In [7]:
def data_stats (data):
    """
    Steps for data preparation
    """
    sent_length = pd.DataFrame(data.document.apply(len).values, columns=['document'])
    sent_length['Summary'] = data.summary.apply(len)
    sent_stats = pd.DataFrame(sent_length.mean().apply(lambda x: round(x,3)), columns = ['mean'])
    #statistics of lengths for each
    sent_length_quant = sent_stats.quantile([0,0.25,0.5,0.75,0.95,0.99, 1])
    sent_length_quant.index.name = 'Percentile'
    sent_length_quant.columns.name = 'Number of Sentences'
    return (sent_stats, sent_length_quant)

In [8]:
print(data_stats(train_df))

(Number of Sentences       mean
document             11017.722
Summary               1300.304, Number of Sentences         mean
Percentile                      
0.00                  1300.30400
0.25                  3729.65850
0.50                  6159.01300
0.75                  8588.36750
0.95                 10531.85110
0.99                 10920.54782
1.00                 11017.72200)


In [9]:
# sent_length = pd.DataFrame(df.document.apply(len).values, columns=['document'])
# sent_length['Summary'] = df.summary.apply(len)
# pd.DataFrame(sent_length.mean().apply(lambda x: round(x,3)), columns = ['mean'])

Article: The mean sentence length is 11k and the median is 8k. The right tail shows some heavy outliers with the 95th to 100th percentile ranging from 52k to 2M.

Summary: The mean is 1.3k and the median is 1.3k also not heavy outliers from 95th to 100th percentile ranging from 2.4k to 5.9k

In [10]:
# import matplotlib.pyplot as plt
# import seaborn as sns
# import numpy as np

# sns.regplot(x='document', y='Summary', data=sent_length);
# plt.title('Regression: Document Length vs Summary Length')
# plt.xlabel('Document Length')
# plt.ylabel('Summary Length')
# r_squared_matrix = np.corrcoef(sent_length.document, sent_length.Summary)**2

# print('r_squared = ', round(r_squared_matrix[0][1],3))

In [11]:
# train_df = train_df.head(50)

In [17]:
! wget http://nlp.stanford.edu/data/glove.6B.zip
! unzip glove*.zip

--2023-11-22 23:25:03--  http://nlp.stanford.edu/data/glove.6B.zip
Resolving nlp.stanford.edu (nlp.stanford.edu)... 171.64.67.140
Connecting to nlp.stanford.edu (nlp.stanford.edu)|171.64.67.140|:80... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://nlp.stanford.edu/data/glove.6B.zip [following]
--2023-11-22 23:25:04--  https://nlp.stanford.edu/data/glove.6B.zip
Connecting to nlp.stanford.edu (nlp.stanford.edu)|171.64.67.140|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: https://downloads.cs.stanford.edu/nlp/data/glove.6B.zip [following]
--2023-11-22 23:25:04--  https://downloads.cs.stanford.edu/nlp/data/glove.6B.zip
Resolving downloads.cs.stanford.edu (downloads.cs.stanford.edu)... 171.64.64.22
Connecting to downloads.cs.stanford.edu (downloads.cs.stanford.edu)|171.64.64.22|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 862182613 (822M) [application/zip]
Saving to: ‘glove.6B.zip’

glov

In [18]:
def data_sent_tokenize(data):
    """
    Sentence tokenize
    """
    sentences = []
    for s in data['document']:
        sentences.append(sent_tokenize(s))
    sentences = [y for x in sentences for y in x]
    return sentences

In [19]:
sentences = data_sent_tokenize(train_df)

In [15]:
def create_glove_embedding():
    """
    Create a dictionary of word embeddings
    """
    word_embeddings = {}
    f = open('glove.6B.100d.txt', encoding='utf-8')
    for line in f:
        values = line.split()
        word = values[0]
        coefs = np.asarray(values[1:], dtype='float32')
        word_embeddings[word] = coefs
    f.close()
    return word_embeddings

In [20]:
word_embeddings = create_glove_embedding()

In [21]:
nltk.download('stopwords')
from nltk.corpus import stopwords
stop_words = stopwords.words('english')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [22]:

# function to remove stopwords
def remove_stopwords(sen):
    sen_new = " ".join([i for i in sen if i not in stop_words])
    return sen_new

In [23]:
def clean_sent(sentences):
    # remove punctuations, numbers and special characters
    clean_sentences = pd.Series(sentences).str.replace("[^a-zA-Z]", " ")
    clean_sentences = pd.Series(sentences).str.replace("|", "")
    # alphabets lowercase
    clean_sentences = [s.lower() for s in clean_sentences]
    clean_sentences = [remove_stopwords(r.split()) for r in clean_sentences]
    return clean_sentences

In [24]:
clean_sentences = clean_sent(sentences)

<ipython-input-23-2f0833459f4d>:3: FutureWarning: The default value of regex will change from True to False in a future version.
  clean_sentences = pd.Series(sentences).str.replace("[^a-zA-Z]", " ")
<ipython-input-23-2f0833459f4d>:4: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  clean_sentences = pd.Series(sentences).str.replace("|", "")


In [ ]:
# # remove punctuations, numbers and special characters
# clean_sentences = pd.Series(sentences).str.replace("[^a-zA-Z]", " ")
# clean_sentences = pd.Series(sentences).str.replace("|", "")

In [25]:
def create_sentence_vectors(clean_sentences):
    sentence_vectors = []
    for i in clean_sentences:
      if len(i) != 0:
        v = sum([word_embeddings.get(w, np.zeros((100,))) for w in i.split()])/(len(i.split())+0.001)
      else:
        v = np.zeros((100,))
      sentence_vectors.append(v)
    return sentence_vectors


In [26]:
len(clean_sentences)

3657863

In [27]:
sentence_vectors = create_sentence_vectors(clean_sentences)

In [38]:
len(sentences), len(sentence_vectors)

(3657863, 3657863)

In [39]:
def similarity_matrix(batch_sentences):
    sim_mat = cosine_similarity(batch_sentences)
    return sim_mat

In [51]:
def batch_similarity_matrix(sentence_vectors):
    sim_matrixes = []
    i = 0
    while i <= int(len(sentences) / 1000):
        if i == int(len(sentences)/1000):
            sim_matrixes.append(cosine_similarity(sentence_vectors[i : len(sentences)]))
        else:
            sim_mat = cosine_similarity(sentence_vectors[i : i + 1000])
            i = i + 1000
            sim_matrixes.append(sim_mat)
    return sim_matrixes

In [52]:
sim_mat = batch_similarity_matrix(sentence_vectors)

0
1000
2000
3000


3657.863

In [54]:
def page_rank_scores(sim_mat):
    nx_graph = nx.from_numpy_array(sim_mat)
    scores = nx.pagerank(nx_graph)
    return scores

In [ ]:
# scores = page_rank_scores(sim_mat)

In [55]:
scores = [page_rank_scores(matrix) for matrix in sim_mat]

In [ ]:
def generate_summary(scores, sentences):
    ranked_sentences = sorted(((scores[i],s) for i,s in enumerate(sentences)), reverse=True)
    summary = [i[1] for i in ranked_sentences[0:10]]
    return summary

train_pred_summary = generate_summary(scores, sentences)

# ranked_sentences = sorted(((scores[i],s) for i,s in enumerate(sentences)), reverse=True)
# # Extract top 10 sentences as the summary
# for i in range(10):
#   print(ranked_sentences[i][1]) # How to compare this with other algorithms.

In [ ]:
def calc_rouge_scores(pred_summaries, gold_summaries,
                                 keys=['rouge1', 'rougeL'], use_stemmer=True):
    #Calculate rouge scores
    scorer = rouge_scorer.RougeScorer(keys, use_stemmer= use_stemmer)

    n = len(pred_summaries)

    scores = [scorer.score(pred_summaries[j], gold_summaries[j]) for
              j in range(n)]

    dict_scores={}
    for key in keys:
        dict_scores.update({key: {}})


    for key in keys:

        precision_list = [scores[j][key][0] for j in range(len(scores))]
        recall_list = [scores[j][key][1] for j in range(len(scores))]
        f1_list = [scores[j][key][2] for j in range(len(scores))]

        precision = np.mean(precision_list)
        recall = np.mean(recall_list)
        f1 = np.mean(f1_list)

        dict_results = {'recall': recall, 'precision': precision, 'f1': f1}

        dict_scores[key] = dict_results

    return dict_scores

In [ ]:
train_df_summary = train_df['summary'].to_list()

In [ ]:
train_rouge_scores = calc_rouge_scores(train_pred_summary, train_df_summary)
train_rouge_scores

In [ ]:
test_df = pd.DataFrame.from_dict(dataset['test'])
test_df = test_df.head(50)

In [ ]:
test_df_summary = test_df['summary'].to_list()

In [ ]:
print(data_stats(test_df))

In [ ]:
sentences = data_sent_tokenize(test_df)
clean_sentences = clean_sent(sentences)
sentence_vectors = create_sentence_vectors(clean_sentences)
sim_mat = similarity_matrix(sentences)
scores = page_rank_scores(sim_mat)
test_pred_summary = generate_summary(scores, sentences)

In [ ]:
test_rouge_scores = calc_rouge_scores(test_pred_summary, test_df_summary)
test_rouge_scores

In [ ]:
import math

from nltk import sent_tokenize, word_tokenize, PorterStemmer
from nltk.corpus import stopwords

In [ ]:
text_str = train_df['document']
text_str = ". ".join(i for i in text_str.to_list())

In [ ]:
def _create_frequency_table(text_string) -> dict:
    """
    we create a dictionary for the word frequency table.
    For this, we should only use the words that are not part of the stopWords array.

    Removing stop words and making frequency table
    Stemmer - an algorithm to bring words to its root word.
    :rtype: dict
    """
    stopWords = set(stopwords.words("english"))
    words = word_tokenize(text_string)
    ps = PorterStemmer()

    freqTable = dict()
    for word in words:
        word = ps.stem(word)
        if word in stopWords:
            continue
        if word in freqTable:
            freqTable[word] += 1
        else:
            freqTable[word] = 1

    return freqTable


def _create_frequency_matrix(sentences):
    frequency_matrix = {}
    stopWords = set(stopwords.words("english"))
    ps = PorterStemmer()

    for sent in sentences:
        freq_table = {}
        words = word_tokenize(sent)
        for word in words:
            word = word.lower()
            word = ps.stem(word)
            if word in stopWords:
                continue

            if word in freq_table:
                freq_table[word] += 1
            else:
                freq_table[word] = 1

        frequency_matrix[sent[:15]] = freq_table

    return frequency_matrix


def _create_tf_matrix(freq_matrix):
    tf_matrix = {}

    for sent, f_table in freq_matrix.items():
        tf_table = {}

        count_words_in_sentence = len(f_table)
        for word, count in f_table.items():
            tf_table[word] = count / count_words_in_sentence

        tf_matrix[sent] = tf_table

    return tf_matrix


def _create_documents_per_words(freq_matrix):
    word_per_doc_table = {}

    for sent, f_table in freq_matrix.items():
        for word, count in f_table.items():
            if word in word_per_doc_table:
                word_per_doc_table[word] += 1
            else:
                word_per_doc_table[word] = 1

    return word_per_doc_table


def _create_idf_matrix(freq_matrix, count_doc_per_words, total_documents):
    idf_matrix = {}

    for sent, f_table in freq_matrix.items():
        idf_table = {}

        for word in f_table.keys():
            idf_table[word] = math.log10(total_documents / float(count_doc_per_words[word]))

        idf_matrix[sent] = idf_table

    return idf_matrix


def _create_tf_idf_matrix(tf_matrix, idf_matrix):
    tf_idf_matrix = {}

    for (sent1, f_table1), (sent2, f_table2) in zip(tf_matrix.items(), idf_matrix.items()):

        tf_idf_table = {}

        for (word1, value1), (word2, value2) in zip(f_table1.items(),
                                                    f_table2.items()):  # here, keys are the same in both the table
            tf_idf_table[word1] = float(value1 * value2)

        tf_idf_matrix[sent1] = tf_idf_table

    return tf_idf_matrix


def _score_sentences(tf_idf_matrix) -> dict:
    """
    score a sentence by its word's TF
    Basic algorithm: adding the TF frequency of every non-stop word in a sentence divided by total no of words in a sentence.
    :rtype: dict
    """

    sentenceValue = {}

    for sent, f_table in tf_idf_matrix.items():
        total_score_per_sentence = 0

        count_words_in_sentence = len(f_table)
        for word, score in f_table.items():
            total_score_per_sentence += score

        sentenceValue[sent] = total_score_per_sentence / count_words_in_sentence

    return sentenceValue


def _find_average_score(sentenceValue) -> int:
    """
    Find the average score from the sentence value dictionary
    :rtype: int
    """
    sumValues = 0
    for entry in sentenceValue:
        sumValues += sentenceValue[entry]

    # Average value of a sentence from original summary_text
    average = (sumValues / len(sentenceValue))

    return average


def _generate_summary(sentences, sentenceValue, threshold):
    sentence_count = 0
    summary = ''

    for sentence in sentences:
        if sentence[:15] in sentenceValue and sentenceValue[sentence[:15]] >= (threshold):
            summary += " " + sentence
            sentence_count += 1

    return summary


def run_summarization(text):
    """
    :param text: Plain summary_text of long article
    :return: summarized summary_text
    """

    '''
    We already have a sentence tokenizer, so we just need
    to run the sent_tokenize() method to create the array of sentences.
    '''
    # 1 Sentence Tokenize
    sentences = sent_tokenize(text)
    total_documents = len(sentences)
    #print(sentences)

    # 2 Create the Frequency matrix of the words in each sentence.
    freq_matrix = _create_frequency_matrix(sentences)
    #print(freq_matrix)

    '''
    Term frequency (TF) is how often a word appears in a document, divided by how many words are there in a document.
    '''
    # 3 Calculate TermFrequency and generate a matrix
    tf_matrix = _create_tf_matrix(freq_matrix)
    #print(tf_matrix)

    # 4 creating table for documents per words
    count_doc_per_words = _create_documents_per_words(freq_matrix)
    #print(count_doc_per_words)

    '''
    Inverse document frequency (IDF) is how unique or rare a word is.
    '''
    # 5 Calculate IDF and generate a matrix
    idf_matrix = _create_idf_matrix(freq_matrix, count_doc_per_words, total_documents)
    #print(idf_matrix)

    # 6 Calculate TF-IDF and generate a matrix
    tf_idf_matrix = _create_tf_idf_matrix(tf_matrix, idf_matrix)
    #print(tf_idf_matrix)

    # 7 Important Algorithm: score the sentences
    sentence_scores = _score_sentences(tf_idf_matrix)
    #print(sentence_scores)

    # 8 Find the threshold
    threshold = _find_average_score(sentence_scores)
    #print(threshold)

    # 9 Important Algorithm: Generate the summary
    summary = _generate_summary(sentences, sentence_scores, 1.3 * threshold)
    return summary


if __name__ == '__main__':
    result = run_summarization(text_str)
    print(result)

In [ ]:
tf_idf_rouge = calc_rouge_scores(result, text_str)
tf_idf_rouge

In [ ]:
train_rouge_scores = calc_rouge_scores(train_pred_summary, train_df_summary)
train_rouge_scores